# Negative Binomial Model Test
This shows that stan, statsmodels and tensorflow have very close results on a simulation dataset.

In [1]:
import sys
sys.path.append("..")
from models.nb import NB
sys.path.remove("..")

import numpy as np

- stan neg_binomial_2_lpmf( k | mu, phi): $(\mu/(\mu+\phi))^k(\phi/(\phi+\mu))^\phi$  
mean $\mu$, variance $\mu+\mu^2/\phi$

- scipy f(k)=nbinom.pmf(k, n, p)  $p^n(1-p)^k$,  
mean $n(1-p)/p$, variance $n(1-p)/p^2$  
scipy: $n=\phi$, $p=\phi/(\phi+\mu)$

- statsmodel: Variance equal to $\mu + \alpha \mu^2$  
statsmodel: $\alpha=1/\phi$


<!-- https://mc-stan.org/docs/2_20/functions-reference/negative-binomial-distribution.html

- stan neg_binomial_lpmf( k | alpha, beta): $(b/(1+b))^a(1/(b+1))^k$, $a/b$, $a/b^2(b+1)$

1-p=1/(b+1), p=b/(b+1), b=1/(1-p)-1=p/(1-p)=alpha, a=



a=n=exb=np.exp(X*beta) -->

In [2]:
from scipy.stats import uniform, binom, nbinom
import statsmodels.api as sm
# Data
np.random.seed(1)                   # set seed to replicate example
nobs= 25000                         # number of obs in model 

x1 = binom.rvs(1, 0.6, size=nobs)   # categorical explanatory variable
x2 = uniform.rvs(size=nobs)         # real explanatory variable

phi = 2
X = sm.add_constant(np.column_stack((x1, x2)))
beta = [1.0, 2.0, -1.5]
xb = np.dot(X, beta)          # linear predictor

exb = np.exp(xb)
nby = nbinom.rvs(phi, phi/(phi+exb))

In [3]:
mod = NB(nby,X,model_path='../models')

In [4]:
res0=mod.fit(method='stan')[0]
res1=mod.fit(method='statsmodels')[0]

true value

In [5]:
list(beta)+[np.log(phi)]

[1.0, 2.0, -1.5, 0.6931471805599453]

In [6]:
res0

{'params': array([ 0.99024041,  2.02096156, -1.51659043,  0.72168813]),
 'llf': -63877.57939102553,
 'df': 4,
 'aic': 127763.15878205106,
 'cpu_time': 0.04229402542114258,
 'model': 'nb',
 'method': 'stan'}

In [7]:
res1

{'params': array([ 0.99076403,  2.01915745, -1.51463773,  0.4859298 ]),
 'llf': -63877.565523685655,
 'df': 4,
 'aic': 127763.13104737131,
 'cpu_time': 0.12883687019348145,
 'model': 'nb',
 'method': 'statsmodels'}

In [8]:
np.exp(res0['params'][-1])*res1['params'][-1]

0.9999970166858148

In [9]:
res0['llf']-res1['llf']

-0.01386733987601474

statsmodels and stan have almost identical results

In [10]:
res2=mod.fit(method='tensorflow')[0]

Metal device set to: Apple M2


In [11]:
res2

{'params': array([ 0.9905465,  2.0188916, -1.5149263,  0.722409 ], dtype=float32),
 'llf': -63877.563322783215,
 'aic': 127763.12664556643,
 'df': 4,
 'cpu_time': 0.7581839561462402,
 'model': 'nb',
 'method': 'tensorflow'}

In [12]:
res0['params']-res2['params']

array([-0.00030611,  0.00206999, -0.00166411, -0.00072088])

In [13]:
res2['llf']-res0['llf']

0.016068242315668613

tensorflow and stan have almost identical results

## Test init

In [14]:
res2=mod.fit(method='stan',start_params=res0['params'])[0]
res3=mod.fit(method='statsmodels',start_params=res1['params'])[0]
res4=mod.fit(method='tensorflow',start_params=res0['params'])[0]

In [15]:
res2

{'params': array([ 0.99060508,  2.01942683, -1.51559369,  0.72163642]),
 'llf': -63877.56952288757,
 'df': 4,
 'aic': 127763.13904577514,
 'cpu_time': 0.045738935470581055,
 'model': 'nb',
 'method': 'stan'}

In [16]:
res3

{'params': array([ 0.99076403,  2.01915745, -1.51463773,  0.4859298 ]),
 'llf': -63877.565523685655,
 'df': 4,
 'aic': 127763.13104737131,
 'cpu_time': 0.012965917587280273,
 'model': 'nb',
 'method': 'statsmodels'}

In [17]:
res4

{'params': array([ 0.99146074,  2.0195456 , -1.5143238 ,  0.71850574], dtype=float32),
 'llf': -63877.625822783215,
 'aic': 127763.25164556643,
 'df': 4,
 'cpu_time': 0.23302030563354492,
 'model': 'nb',
 'method': 'tensorflow'}

In [18]:
mod = NB(np.array([nby,nby]).T,X,model_path='../models')

In [19]:
res5=mod.fit(method='tensorflow',start_params=res0['params'])
res6=mod.fit(method='tensorflow',start_params=[r['params'] for r in res5])
res7=mod.fit(method='stan',start_params=[r['params'] for r in res5])

In [20]:
res5

[{'params': array([ 0.9910591 ,  2.0194516 , -1.5143504 ,  0.72110856], dtype=float32),
  'llf': -63879.750822783215,
  'aic': 127767.50164556643,
  'df': 4,
  'cpu_time': 0.15061652660369873,
  'model': 'nb',
  'method': 'tensorflow'},
 {'params': array([ 0.9910591 ,  2.0194516 , -1.5143504 ,  0.72110856], dtype=float32),
  'llf': -63879.750822783215,
  'aic': 127767.50164556643,
  'df': 4,
  'cpu_time': 0.15061652660369873,
  'model': 'nb',
  'method': 'tensorflow'}]

In [21]:
res6

[{'params': array([ 0.99007946,  2.018473  , -1.5153286 ,  0.72096896], dtype=float32),
  'llf': -63878.938322783215,
  'aic': 127765.87664556643,
  'df': 4,
  'cpu_time': 0.1254035234451294,
  'model': 'nb',
  'method': 'tensorflow'},
 {'params': array([ 0.99007946,  2.018473  , -1.5153286 ,  0.72096896], dtype=float32),
  'llf': -63878.938322783215,
  'aic': 127765.87664556643,
  'df': 4,
  'cpu_time': 0.1254035234451294,
  'model': 'nb',
  'method': 'tensorflow'}]

In [22]:
res7

[{'params': array([ 0.99071271,  2.01915828, -1.51452891,  0.72169065]),
  'llf': -63877.56554087238,
  'df': 4,
  'aic': 127763.13108174477,
  'cpu_time': 0.04932117462158203,
  'model': 'nb',
  'method': 'stan'},
 {'params': array([ 0.99071271,  2.01915828, -1.51452891,  0.72169065]),
  'llf': -63877.56554087238,
  'df': 4,
  'aic': 127763.13108174477,
  'cpu_time': 0.04902005195617676,
  'model': 'nb',
  'method': 'stan'}]